In [1]:
import os
import json
import pandas as pd
from datetime import datetime, timezone

In [48]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflows'

In [49]:
os.chdir("C:/Users/pc/Desktop/GitHub repos/End-to-end-ML-project-with-MLflows")

In [50]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflows'

In [51]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [52]:
from mlProject.constants.__init__ import *
from mlProject.utils.common import read_yaml, create_dirs

In [53]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

In [54]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [55]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: /n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [57]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-19 15:37:37,539: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-19 15:37:37,543: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-19 15:37:37,547: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-19 15:37:37,549: INFO: common: created directory at: artifacts]
[2025-04-19 15:37:37,554: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-19 15:38:23,687: INFO: 2123895545: artifacts/data_ingestion/bike_stations.zip download! with following info: /nConnection: close
Content-Length: 236010999
Content-Type: application/octet-stream
Last-Modified: Tue, 07 Dec 2021 09:05:44 GMT
ETag: "0x8D9B960C0848380"
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-request-id: 879b8066-f01e-004c-1d87-b07165000000
x-ms-version: 2025-01-05
x-ms-creation-time: Tue, 17 Aug 2021 07:46:27 GMT
x-ms-lease-status: unlocked
x-ms-lease-state: available
x-ms-blob-type: BlockBlob
Content-Disposition: attachment; filename=